In [1]:
# Connect to local database
from cytopy.data.setup import global_init
global_init("cytopy_db")
# CytoPy imports
from cytopy.data.project import Project
from cytopy.data.read_write import get_fcs_file_paths
import pandas as pd

<h2>Create project</h2>

Start by creating the peritonitis `Project`

In [2]:
#HDF5 files will be stored locally at /media/ross/SP PHD U3/PD_CytoPy
peritonitis = Project(project_id="Peritonitis", data_directory="/media/ross/SP PHD U3/PD_CytoPy")
peritonitis.save()

In [2]:
peritonitis = Project.objects(project_id="Peritonitis").get()

<h2>Add subjects</h2>

In [7]:
# Load patient meta data as a Pandas DataFrame
patient_meta = pd.read_csv("patient_categories.csv")

For each patient I'm going to create a new `Subject` associated to my peritonitis project using the `add_subject` method. The subject will contain the fields: status and infection_data. Infectious data is a embedded field with the additional fields bacterial_isolate and gram_status. This is easy to create by just passing a dictionary. The subject is dynamic and can accept complex meta data to accomodate different project designs a

In [5]:
patients = patient_meta.study_id.values
for p in patients:
    meta = patient_meta[patient_meta.study_id == p].to_dict("records")[0]
    peritonitis.add_subject(subject_id=p,
                            status=meta.get("status"),
                            infection_data={"bacterial_isolate": meta.get("bacterial_isolate"),
                                            "gram_status": meta.get("gram_status")})

<h2>Create experiment and add files</h2>

In [6]:
# Add an experiment with to house T cells from PBMCs. The staining definition is 
# provided as an excel template
peritonitis.add_experiment(experiment_id="PBMC T cells", panel_definition="panels/T_Panel.xlsx")

<Experiment: Experiment object>

In [3]:
t_cells_pbmc = peritonitis.load_experiment("PBMC T cells")

The function below is custom to these project but makes use of the `get_fcs_file_paths`, a utility function in CytoPy that makes locating fcs files easier. This function takes a default path where I know I have the FCS files stored on my local drive. The function takes a list of control file names to expect which is passed to `control_names`. For this projects control files are identified by containing the term "FMO" in their filename, so I can tell CytoPy to find them with the `ctrl_id` argument. I'm also not interested in the compensation files (compensation was performed using FACSDiva software, embedding a spillover matrix into the FCS files).

If the primary staining file is missing (identified by the absence of the "FMO" term in the filename) or there are multiple primary stianing files, the function will print a warning statement and return `None`. Otherwise, the file path to the primary staining file will be return along with a dictionary of control filepaths.

In [4]:
def filepaths(pt_id: str, ctrls: list):
    """
    Given a patient identifier from the Peritonitis project, search the local data 
    repository for fcs files. Locate the primary staining and any controls (if provided).
    
    Returns
    -------
    str, dict
        Path to primary staining, dictionary of control staining file paths
    """
    dirpath = f"/home/ross/CytoPyProjects/CytoPy_Manuscript/02 Validation/setup/data/{pt_id}"
    filepaths = get_fcs_file_paths(dirpath,
                                   control_names=ctrls,
                                   ctrl_id="FMO",
                                   ignore_comp=True,
                                   exclude_dir="DUPLICATE")
    if len(filepaths.get("primary")) == 0:
        print(f"WARNING: no primary files found for {pt_id} in {panel}")
        return None, None
    if len(filepaths.get("primary")) > 1:
        print(f"WARNING: multiple primary files found for {pt_id} in {panel}")
        return None, None
    primary = filepaths.get("primary")[0]
    controls = filepaths.get("controls")
    if len(controls) > 0:
        controls = {x: v[0] for x, v in controls.items()}
    return primary, controls

Iterate over each patient and populate the experiment using fcs files.

In [5]:
t_ctrls = ["CXCR3", "CD25", "CD45RA", "CCR7"]

In [10]:
for pt_id in patients:
    print(f"======== {pt_id} ========")
    primary, controls = filepaths(pt_id, t_ctrls)
    if primary is not None:
        t_cells_pbmc.add_fcs_files(sample_id=pt_id,
                                   primary=primary,
                                   controls=controls,
                                   subject_id=pt_id,
                                   compensate=True,
                                   verbose=False,
                                   missing_error="warn")
    
    print("==========================\n")

======== 165-09 ========


/home/ross/CytoPy/CytoPy/data/experiment.py:372: UserWarning: Missing channel FSC-W
  warn(f"Missing channel {x.standard}")



======== 175-09 ========

======== 210-14 ========

======== 251-07 ========

======== 254-05 ========

======== 255-04 ========

======== 264-02 ========

======== 273-01 ========

======== 286-04 ========

======== 294-02 ========


/home/ross/CytoPy/CytoPy/data/experiment.py:372: UserWarning: Missing channel SSC-H
  warn(f"Missing channel {x.standard}")



======== 305-02 ========

======== 306-01 ========

======== 310-01 ========

======== 318-01 ========

